In [7]:
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization, traduci_output, process_text_with_models, process_emotions_and_translate

### carico i modelli

In [ ]:
# 1. Recupera i dettagli dei modelli
model_details = get_model_details()

# 2. Assegna manualmente le tuple a  separate
(model_1_id, model_1_type, model_1_files), (model_2_id, model_2_type, model_2_files), (model_3_id, model_3_type, model_3_files), (model_4_id, model_4_type, model_4_files) = model_details

# 3. Usa i dettagli per caricare i modelli
print(f"Caricamento modello 1: {model_1_id}")
model_1_pipeline = load_model_and_files(model_1_id, model_type=model_1_type)

print(f"Caricamento modello 2: {model_2_id}")
model_2_pipeline = load_model_and_files(model_2_id, model_type=model_2_type)

print(f"Caricamento modello 3: {model_3_id}")
model_3_pipeline = load_model_and_files(model_3_id, model_type=model_3_type)

print(f"Caricamento modello 4: {model_4_id}")
model_4_pipeline = load_model_and_files(model_4_id, model_type=model_4_type)


Caricamento modello 1: osiria/bert-italian-uncased-ner


Device set to use cpu


Caricamento modello 2: IVN-RIN/MedPsyNIT


Device set to use cpu


Caricamento modello 3: SamLowe/roberta-base-go_emotions


Device set to use cpu


Caricamento modello 4: Helsinki-NLP/opus-mt-it-en


In [21]:
text = "Ciao Lorenzo, ho scoperto una cosa che davvero mi ha sconvolto. Ho parlato con Giada a Bologna, e mi ha detto che l'altra sera ti ha visto al bar con un'altra donna. Io davvero sono senza parole, pretendo delle spiegazioni. Mi sono fidata di te, anche dopo che mi hai rassicurato più e più volte sulla tua fedeltà nei miei confronti. Ora basta. Ti voglio fuori dalla mia vita e dalla mia casa. L'hai tanto denigrata, finalmente sei libero di trovarne una tua come piace a te. PS: fottiti."

### faccio una prova utilizzando direttamente i modelli sul testo crudo

In [18]:

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Ciao Lorenzo, ho scoperto una cosa che davvero mi ha sconvolto. Ho parlato con Giada a Bologna, e mi ha detto che l'altra sera ti ha visto al bar con un'altra donna. Io davvero sono senza parole, pretendo delle spiegazioni. Mi sono fidata di te, anche dopo che mi hai rassicurato più e più volte sulla tua fedeltà nei miei confronti. Ora basta. Ti voglio fuori dalla mia vita e dalla mia casa. L'hai tanto denigrata, finalmente sei libero di trovarne una tua come piace a te. PS: fottiti.

Risultati finali dopo unione dei modelli:
Parola: lorenzo, Entità: PER, Score: 0.9657
--------------------------------
Parola: giada, Entità: PER, Score: 0.9027
--------------------------------
Parola: bologna, Entità: LOC, Score: 0.9925
--------------------------------


### faccio una prova puleno il testo

In [4]:
# 3. Definisci il testo su cui eseguire l'analisi

text_processed = preprocess_text(text)
text_processed = preprocess_stopwords(text_processed)
text_processed = preprocess_lemmatization(text_processed)

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print(text_processed)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie
volere sapere Giulia vivere Milano c ' serio speranza problema serio endometrioso fibrome uterino raggiungere il 50 . consapevole anzianotta , speranza c ' .

Risultati finali dopo unione dei modelli:
Parola: giulia, Entità: MISC, Score: 0.3940
--------------------------------
Parola: milano, Entità: MISC, Score: 0.5187
--------------------------------
Parola: speranza, Entità: MISC, Score: 0.3922
--------------------------------
Parola: endometrioso, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.4980
--------------------------------
Parola: fime, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.5342
--------------------------------
Parola: uterino, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.6979
--------------------------------


### faccio sentiment Analys

In [22]:
# Tradurre il testo con il modello di traduzione
text_tradotto = model_4_pipeline(text)

# Classificare le emozioni nel testo tradotto
output_classificazione = model_3_pipeline(text_tradotto)

# Tradurre i label delle emozioni in italiano
output_tradotto = traduci_output(output_classificazione[0])

In [23]:
output_tradotto

[{'label': 'gioia', 'score': 0.5552638173103333},
 {'label': 'ammirazione', 'score': 0.28733378648757935},
 {'label': 'divertimento', 'score': 0.1629708707332611},
 {'label': 'orgoglio', 'score': 0.13074733316898346},
 {'label': 'eccitazione', 'score': 0.05566107854247093},
 {'label': 'approvazione', 'score': 0.048677101731300354},
 {'label': 'gratitudine', 'score': 0.042775001376867294},
 {'label': 'rilievo', 'score': 0.03504181653261185},
 {'label': 'neutrale', 'score': 0.029221637174487114},
 {'label': 'realizzazione', 'score': 0.02292788401246071},
 {'label': 'fastidio', 'score': 0.01711045578122139},
 {'label': 'ottimismo', 'score': 0.013466647826135159},
 {'label': 'imbarazzo', 'score': 0.00791418831795454},
 {'label': 'cura', 'score': 0.006322175730019808},
 {'label': 'sorpresa', 'score': 0.005588737316429615},
 {'label': 'disapprovazione', 'score': 0.005490300711244345},
 {'label': 'desiderio', 'score': 0.005143625661730766},
 {'label': 'delusione', 'score': 0.00455998862162232